In [11]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

In [12]:
!pip install spotipy

In [38]:
os.environ['SPOTIPY_CLIENT_ID'] = ''
os.environ['SPOTIPY_CLIENT_SECRET'] = ''

In [39]:
# spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

client_credentials_manager = SpotifyClientCredentials(
    client_id=os.environ['SPOTIPY_CLIENT_ID'],
    client_secret=os.environ['SPOTIPY_CLIENT_SECRET']
)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [40]:
# Get details of a specific playlist
def get_playlist_details(spotify,playlist_id):
    playlist_details = []
    offset = 0
    while offset<10000:
        items = spotify.playlist_items(playlist_id,market='US',offset=offset)["items"]
        playlist_details = playlist_details + items
        offset+=100
    return playlist_details

In [62]:
# Replace the playlist id if you want to extract songs from a different playlist
playlist_id = '1G8IpkZKobrIlXcVPoSIuf'
playlist_details = get_playlist_details(spotify,playlist_id)

In [63]:
# Fetch track ids residing within a playlist
def get_track_ids(playlist):
    track_ids = []
    for track in playlist_details:
        if track["track"]:
            if track["track"]["id"]:
                track_ids.append(track["track"]["id"])
    return track_ids

In [64]:
track_ids = get_track_ids(playlist_details)

In [65]:
# Get details of a track
def get_track_details(track_ids):
    tracks = []
    artists = []
    count=0
    for track_id in track_ids:
        track = spotify.track(track_id)
        track_dic = {}

        artists.append([artist['id'] for artist in track['artists']])
        track_dic['track_name'] = track['name']
        track_dic['track_popularity'] = track['popularity']
        track_dic['album_released_date'] = track['album']['release_date']
        track_dic['id'] = track['id']
        track_dic['album_id'] = track['album']['id']
        track_dic['album_name'] = track['album']['name']
        track_dic['album_type'] = track['album']['type']
        tracks.append(track_dic)
        count+=1
        print(count)
    return (tracks,artists)

In [ ]:
tracks,artist_ids = get_track_details(track_ids)

In [48]:
tracks_truncated.head()

NameError: name 'tracks_truncated' is not defined

In [52]:
tracks_truncated = pd.DataFrame.from_records(tracks)

NameError: name 'tracks' is not defined

In [17]:
tracks_truncated['artist_id'] = artist_ids

In [18]:
tracks_truncated.to_csv('../data/truncated_tracks.csv')

In [4]:
tracks_truncated = pd.read_csv('../data/truncated_tracks.csv',index_col=0)

In [13]:
tracks_truncated['artist_id'] = tracks_truncated['artist_id'].apply(eval)

In [12]:
# Fetching artist details for a track
def get_artist_details_for_track(spotify,artists_ids,track_ids):
    
    # artist information to be returned
    genres = []
    popularity = []
    artist_names = []
    
    # batch arrays acting as a buffer for the output of aritist API
    artists_batch = []
    group_counter = 0
    track_id_batch = []
    
    for artist_id_group,track_id,index in zip(artists_ids,track_ids,list(range(len(artists_ids)))):
        
        #check if the length of a batch does not exceed 50 (maximum api batch size input constraint)
        if ((len(artist_id_group) + len(artists_batch)) <= 50 and (index<len(artists_ids)-1)):
            group_counter += 1
            artists_batch = artists_batch + artist_id_group
            track_id_batch.append(track_id)
        else:
            if index==len(artists_ids)-1:
                artists_batch = artists_batch + artist_id_group
        
            # API call to the artist function
            artist_details = spotify.artists(artists_batch)
            offset = 0
            
            # extracting artist details and appending it to the output arrays
            for group_index in range(group_counter):
                group_size = len(artists_ids[group_index])
                genres.append((([artist['genres'] for artist in artist_details['artists'][offset:offset+group_size]]),track_id_batch[group_index]))
                popularity.append((([artist['popularity'] for artist in artist_details['artists'][offset:offset+group_size]]),track_id_batch[group_index]))
                artist_names.append((([artist['name'] for artist in artist_details['artists'][offset:offset+group_size]]),track_id_batch[group_index]))
                offset+=group_size
            
            # resetting batch parameters for the next bacth
            artists_batch=[]
            track_id_batch=[]
            group_counter = 0
    
    return (genres,popularity,artist_names)

In [14]:
artist_ids = list(tracks_truncated['artist_id'])
track_ids = list(tracks_truncated['id'])
genres,popularity,artist_names = get_artist_details_for_track(spotify,artist_ids,track_ids)

In [15]:
artist_names[0:60]

[(['Bob Dylan'], '3AhXZa8sUQht0UEdBJgpGc'),
 (['Nirvana'], '5ghIJDpPoe3CfHMGu71E6T'),
 (['The Beatles'], '0hKRSZhUGEhKU6aNSPBACZ'),
 (['The Beach Boys'], '5Qt4Cc66g24QWwGP3YYV9y'),
 (['Chuck Berry'], '2QfiRTz5Yc8DdShCxG1tB2'),
 (['Aretha Franklin'], '7s25THrKz86DM225dOYwnr'),
 (['The Ronettes'], '2G2YzndIA6jeWFPBXhUjh5'),
 (['Marvin Gaye'], '3Um9toULmYFGCpvaIPFw7l'),
 (['Marvin Gaye'], '1tqT6DhmsrtQgyCKUwotiw'),
 (['The Rolling Stones'], '2PzU4IB8Dr6mxV3lHuaG34'),
 (['Michael Jackson'], '5ChkMS8OtdzJeqyybCc9R5'),
 (['Elvis Presley'], '1iUgrte2PxLxQ6Te95RnAV'),
 (['Bruce Springsteen'], '6hTcuIQa0sxrrByu9wTD7s'),
 (['The Who'], '4u9f8hqstB7iITDJNzKhQx'),
 (['Joy Division'], '34iOH7LY3vme5rQxsVILZ4'),
 (['The Beach Boys'], '6iGU74CwXuT4XVepjc9Emf'),
 (['Otis Redding'], '4Y4Gd3ty8uut6Qw43c7yJc'),
 (['Outkast'], '2PpruBYCo4H7WOBJ7Q2EwM'),
 (['Sex Pistols'], '3LoDeIbiR12sAznmpNEmKA'),
 (['M.I.A.'], '1ixbwbeBi5ufN4noUKmW5a'),
 (['The Clash'], '5jzma6gCzYtKB1DbEwFZKH'),
 (['The Rolling Stones'

In [16]:
genre_df = pd.DataFrame(genres, columns =['genre', 'track_id'])

In [19]:
popularity_df = pd.DataFrame(popularity, columns =['popularity', 'track_id'])

In [20]:
artist_names_df = pd.DataFrame(artist_names, columns =['artist_names', 'track_id'])

In [21]:
temp = pd.concat([artist_names_df,genre_df.drop(columns=['track_id'])],axis=1)

In [22]:
artist_details_df = pd.concat([temp,popularity_df.drop(columns=['track_id'])],axis=1)

In [23]:
artist_details_df

,artist_names,track_id,genre,popularity
0,[Bob Dylan],3AhXZa8sUQht0UEdBJgpGc,"[[album rock, classic rock, country rock, folk...",[71]
1,[Nirvana],5ghIJDpPoe3CfHMGu71E6T,"[[alternative rock, grunge, permanent wave, ro...",[81]
2,[The Beatles],0hKRSZhUGEhKU6aNSPBACZ,"[[beatlesque, british invasion, classic rock, ...",[83]
3,[The Beach Boys],5Qt4Cc66g24QWwGP3YYV9y,"[[baroque pop, brill building pop, classic roc...",[72]
4,[Chuck Berry],2QfiRTz5Yc8DdShCxG1tB2,"[[classic rock, rock, rock-and-roll, rockabilly]]",[65]
...,...,...,...,...
9727,[Animotion],5iyCSUM7zzficwaGo8GIoc,"[[new romantic, new wave, new wave pop, synthp...",[42]
9728,[Chris Montez],2QSD3K3b3BJ8DPhGhQfDPW,"[[adult standards, brill building pop, bubbleg...",[46]
9729,[The Beatles],67mtN6BlsJR0LVyzvNZnGt,"[[beatlesque, british invasion, classic rock, ...",[83]
9730,[Skids],1KcxkWlRSDNchLovHnHBKP,"[[new wave, pub rock, punk, scottish new wave,...",[35]


In [25]:
artist_details_df.to_csv('../data/artist_details.csv')

In [26]:
artist_details_df = pd.read_csv('../data/artist_details.csv',index_col=0)

In [27]:
artist_details_df['track_id'].value_counts()

3qdDjz9kOvquEFbvXBYOMQ    1
5RNOPiFhFVyTpoPmm0XNCa    1
44KIBTGQket3oXHFIGvvIw    1
7FIhuHZUdUxWX2hDMGdIbF    1
722E6CPY8biq4sUnf40u6Y    1
                         ..
2EFtUP7BmsjHzhQqI6A4rp    1
1qG6DKMmMrXMIC75dXy5r0    1
5NK8jad728pj6YeqM5VJD4    1
78kbSMbALAqKFAJ7In4lyf    1
1kG2PZ8geznbDA8I6iWeDi    1
Name: track_id, Length: 9732, dtype: int64

In [31]:
# Fetch audio features of tracks
def get_track_audio_features(spotify,track_ids):
    audio_features = []
    offset = 0
    while offset<len(track_ids):
        result = spotify.audio_features(track_ids[offset:offset+100])
        audio_features = audio_features + result
        offset+=100
    return audio_features

In [33]:
track_audio_features = get_track_audio_features(spotify,track_ids)

In [35]:
track_audio_features_df = pd.DataFrame.from_records(track_audio_features)

In [40]:
track_audio_features_df.to_csv('../data/track_audio_features.csv')